In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# CovidActNow API Key
from config import api_key
# Census API Key
from config import census_key

In [2]:
# Create state population dataframe

# Specify the Census URL
census_url = "https://api.census.gov/data/2021/pep/population?"
# Build Census Pops by state query URL
census_query_url = f"{census_url}get=DENSITY_2021,POP_2021,NAME,STATE&for=state:*&key={census_key}"

# Make state pops data request and save to a dataframe
pops_response = requests.get(census_query_url)
pops_json = pops_response.json()
row_counter = len(pops_json)
pops_data = []
for row in range(1, row_counter):
    pops_data.append(pops_json[row])
pops_data_df = pd.DataFrame(pops_data, columns = pops_json[0])

# Create new dataframe with only the needed data columns
reduced_pops_data_df = pops_data_df[["NAME", "POP_2021"]]

# Create dataframe that contains state name and ID
state_id = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY",
            "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
            "OK", "OR", "PA", "PR", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
state_list = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", 
              "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
              "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
              "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", 
              "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", 
              "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
              "West Virginia", "Wisconsin", "Wyoming"]
state_id_df = pd.DataFrame({"NAME" : state_list,
                            "state" : state_id})

# Combine the data into a single dataset
state_pops_data_df = pd.merge(state_id_df, reduced_pops_data_df, on = "NAME", how = "left", sort = False)

In [3]:
# Display preview of state population dataframe
state_pops_data_df.head()

,NAME,state,POP_2021
0,Alabama,AL,5039877
1,Alaska,AK,732673
2,Arizona,AZ,7276316
3,Arkansas,AR,3025891
4,California,CA,39237836


In [4]:
# Create Covid dataframe

# Specify the Covid URL
covid_url = "https://api.covidactnow.org/v2/states.timeseries.csv?apiKey="
# Build CovidActNow query URL
covid_query_url = f"{covid_url}{api_key}"

# Make Covid data request and save to a CSV file
covid_url_response = requests.get(covid_query_url)
covid_url_content = covid_url_response.content
csv_file = open('covid_data.csv', 'wb')
csv_file.write(covid_url_content)
csv_file.close()

# Path to the Covid data file
covid_data_path = "covid_data.csv"
# Read the CSV file and create a dataframe
covid_data_df = pd.read_csv(covid_data_path)

In [5]:
# Merge the covid and state population dataframes
covid_data_df = pd.merge(covid_data_df, state_pops_data_df, on = "state", how = "left", sort = False)

In [6]:
# Display merged dataframe
covid_data_df

,date,country,state,county,fips,lat,long,locationId,actuals.cases,actuals.deaths,...,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,riskLevels.caseDensity,cdcTransmissionLevel,actuals.vaccinationsAdditionalDose,metrics.vaccinationsAdditionalDoseRatio,NAME,POP_2021
0,2020-03-01,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Alaska,732673
1,2020-03-02,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Alaska,732673
2,2020-03-03,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Alaska,732673
3,2020-03-04,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Alaska,732673
4,2020-03-05,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Alaska,732673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37274,2022-01-26,US,WY,NaN,56,NaN,NaN,iso1:us#iso2:us-wy,141090.0,1625.0,...,0.571,0.497,0.0,712362.0,5,3,114447.0,0.198,Wyoming,578803
37275,2022-01-27,US,WY,NaN,56,NaN,NaN,iso1:us#iso2:us-wy,143129.0,1625.0,...,0.571,0.497,0.0,713287.0,5,3,114925.0,0.199,Wyoming,578803
37276,2022-01-28,US,WY,NaN,56,NaN,NaN,iso1:us#iso2:us-wy,144526.0,1625.0,...,0.572,0.498,0.0,714237.0,5,3,115443.0,0.199,Wyoming,578803
37277,2022-01-29,US,WY,NaN,56,NaN,NaN,iso1:us#iso2:us-wy,NaN,1625.0,...,0.572,0.498,0.0,715276.0,5,3,116007.0,0.200,Wyoming,578803


In [7]:
# Create new dataframe with only the needed data columns and date range
clean_covid_df = covid_data_df[["date" , 
                                "state", 
                                "actuals.cases", 
                                "actuals.newCases", 
                                "actuals.deaths", 
                                "actuals.newDeaths", 
                                "actuals.positiveTests", 
                                "actuals.negativeTests",  
                                "actuals.hospitalBeds.capacity", 
                                "actuals.hospitalBeds.currentUsageCovid",
                                "actuals.icuBeds.capacity", 
                                "actuals.icuBeds.currentUsageCovid",
                                "actuals.vaccinesDistributed", 
                                "actuals.vaccinesAdministered",
                                "actuals.vaccinationsInitiated", 
                                "actuals.vaccinationsCompleted",  
                                "actuals.vaccinationsAdditionalDose",
                                "POP_2021"]]

# Rename columns for easier viewing
clean_covid_df = clean_covid_df.rename(columns={"date" : "Date", 
                                                "state" : "State", 
                                                "actuals.cases" : "Total_Cases",
                                                "actuals.newCases" : "New_Cases", 
                                                "actuals.deaths" : "Total_Deaths",
                                                "actuals.newDeaths" : "New_Deaths", 
                                                "actuals.positiveTests" : "Total_Positives",
                                                "actuals.negativeTests" : "Total_Negatives", 
                                                "actuals.hospitalBeds.capacity": "Total_Beds",
                                                "actuals.hospitalBeds.currentUsageCovid" : "Current_Bed",
                                                "actuals.icuBeds.capacity" : "Total_ICU", 
                                                "actuals.icuBeds.currentUsageCovid" : "Current_ICU",
                                                "actuals.vaccinesDistributed" : "Vaccine_Dist", 
                                                "actuals.vaccinesAdministered" : "Vaccine_Admin",
                                                "actuals.vaccinationsInitiated" : "Vaccine_Int", 
                                                "actuals.vaccinationsCompleted" : "Vaccine_Complete",
                                                "actuals.vaccinationsAdditionalDose" : "Add_Dose",
                                                "POP_2021" : "Population"})

# Filter for the data from 12/31/20 to most recent data
clean_covid_df = clean_covid_df.loc[(clean_covid_df["Date"] > "2020-12-30") &
                                    (clean_covid_df["Date"] < "2022-01-01"), :].reset_index(drop=True)

# Create "Year", "Month" and "Day" columns based on the "Date" column
clean_covid_df["Year"] = pd.DatetimeIndex(clean_covid_df["Date"]).year
clean_covid_df["Month"] = pd.DatetimeIndex(clean_covid_df["Date"]).month
clean_covid_df["Day"] = pd.DatetimeIndex(clean_covid_df["Date"]).day

# Move "Year", "Month" and "Day" columns to the front of the dataframe
year_column = clean_covid_df.pop("Year")
month_column = clean_covid_df.pop("Month")
day_column = clean_covid_df.pop("Day")
clean_covid_df.insert(0, "Year", year_column)
clean_covid_df.insert(1, "Month", month_column)
clean_covid_df.insert(2, "Day", day_column)

# Replace NaN values with 0
clean_covid_df = clean_covid_df.fillna(0)

In [8]:
# Display dataframe created above
clean_covid_df

,Year,Month,Day,Date,State,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Total_Positives,...,Total_Beds,Current_Bed,Total_ICU,Current_ICU,Vaccine_Dist,Vaccine_Admin,Vaccine_Int,Vaccine_Complete,Add_Dose,Population
0,2020,12,31,2020-12-31,AK,46740.0,499.0,198.0,3.0,84167.0,...,1642.0,80.0,127.0,10.0,54975.0,14550.0,0.0,0.0,0.0,732673
1,2021,1,1,2021-01-01,AK,46740.0,0.0,198.0,0.0,84423.0,...,1643.0,78.0,128.0,10.0,54975.0,16504.0,0.0,0.0,0.0,732673
2,2021,1,2,2021-01-02,AK,47549.0,809.0,207.0,9.0,84687.0,...,1641.0,88.0,127.0,11.0,54975.0,18401.0,0.0,0.0,0.0,732673
3,2021,1,3,2021-01-03,AK,47846.0,297.0,207.0,0.0,85010.0,...,1643.0,98.0,127.0,13.0,54975.0,18684.0,0.0,0.0,0.0,732673
4,2021,1,4,2021-01-04,AK,48110.0,264.0,209.0,2.0,85457.0,...,1447.0,86.0,127.0,16.0,75475.0,18937.0,0.0,0.0,0.0,732673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2021,12,27,2021-12-27,WY,114624.0,382.0,1526.0,0.0,88859.0,...,1598.0,69.0,140.0,25.0,846065.0,679449.0,321856.0,273926.0,95682.0,578803
19394,2021,12,28,2021-12-28,WY,114917.0,293.0,1526.0,0.0,89052.0,...,1598.0,73.0,140.0,21.0,846065.0,681822.0,322399.0,274405.0,97023.0,578803
19395,2021,12,29,2021-12-29,WY,115242.0,325.0,1526.0,0.0,89294.0,...,1743.0,80.0,140.0,25.0,846565.0,683069.0,322765.0,274666.0,97669.0,578803
19396,2021,12,30,2021-12-30,WY,115638.0,396.0,1526.0,0.0,89578.0,...,1743.0,69.0,140.0,24.0,845705.0,684672.0,323245.0,275046.0,98480.0,578803


In [9]:
# Create monthly set 1 dataframe
monthly_set1_df = clean_covid_df[["State",
                                  "Year", 
                                  "Month", 
                                  "Day", 
                                  "Population",
                                  "Total_Cases", 
                                  "Total_Deaths", 
                                  "Total_Positives",
                                  "Total_Negatives", 
                                  "Vaccine_Dist",
                                  "Vaccine_Admin", 
                                  "Vaccine_Int",
                                  "Vaccine_Complete", 
                                  "Add_Dose"]]

# Sort the data using "State"(ascending), "Year"(ascending), "Month"(ascending)  and "Day" (descending) columns 
sorted_monthly_set1_df = monthly_set1_df.sort_values(by = ["State", "Year", "Month", "Day"], 
                                                     ascending = [True, True, True, False])

# Keep only the data for the last day of each month, remove all other days
final_monthly_set1_df = sorted_monthly_set1_df.drop_duplicates(subset = ["State", "Year", "Month"],
                                                               keep = "first").reset_index(drop=True)

# Define columns for calculated monthly values and state population
final_monthly_set1_df[["Monthly Cases", "Monthly Deaths", "Monthly Positive", "Monthly Negatives",
                       "Monthly Vaccine Initiated", "Monthly Vaccine Completed", "Monthly Additional Dose",
                       "Year-Month"]] = ""

# Populate monthly columns with calculated values 
row_counter = len(final_monthly_set1_df)
row_counter

for x in range(1, row_counter):
    final_monthly_set1_df.iloc[x, 14] = final_monthly_set1_df.iloc[x, 5] - final_monthly_set1_df.iloc[x-1, 5]
    final_monthly_set1_df.iloc[x, 15] = final_monthly_set1_df.iloc[x, 6] - final_monthly_set1_df.iloc[x-1, 6]
    final_monthly_set1_df.iloc[x, 16] = final_monthly_set1_df.iloc[x, 7] - final_monthly_set1_df.iloc[x-1, 7]
    final_monthly_set1_df.iloc[x, 17] = final_monthly_set1_df.iloc[x, 8] - final_monthly_set1_df.iloc[x-1, 8]
    final_monthly_set1_df.iloc[x, 18] = final_monthly_set1_df.iloc[x, 11] - final_monthly_set1_df.iloc[x-1, 11]
    final_monthly_set1_df.iloc[x, 19] = final_monthly_set1_df.iloc[x, 12] - final_monthly_set1_df.iloc[x-1, 12]
    final_monthly_set1_df.iloc[x, 20] = final_monthly_set1_df.iloc[x, 13] - final_monthly_set1_df.iloc[x-1, 13]
    final_monthly_set1_df.iloc[x, 21] = f"{final_monthly_set1_df.iloc[x, 1]}-{final_monthly_set1_df.iloc[x, 2]}"

In [10]:
# Clean up monthly set1 dataframe for easy viewing

# Remove all rows containing 12/31/2020 data
final_monthly_set1_df = final_monthly_set1_df.loc[final_monthly_set1_df["Year"] != 2020, :]

# Remove columns not needed
final_monthly_set1_df = final_monthly_set1_df.drop(columns = ["Year","Month", "Day"])

# Move the "Month-Year" column after the state ID
year_month = final_monthly_set1_df.pop("Year-Month")
final_monthly_set1_df.insert(1, "Year-Month", year_month)

In [11]:
# Display the final monthly set1 data
final_monthly_set1_df

,State,Year-Month,Population,Total_Cases,Total_Deaths,Total_Positives,Total_Negatives,Vaccine_Dist,Vaccine_Admin,Vaccine_Int,Vaccine_Complete,Add_Dose,Monthly Cases,Monthly Deaths,Monthly Positive,Monthly Negatives,Monthly Vaccine Initiated,Monthly Vaccine Completed,Monthly Additional Dose
1,AK,2021-1,732673,53985.0,253.0,93699.0,1547506.0,193150.0,119590.0,0.0,0.0,0.0,7245.0,55.0,9532.0,226096.0,0.0,0.0,0.0
2,AK,2021-2,732673,57846.0,279.0,98919.0,1742031.0,342535.0,264741.0,166735.0,0.0,0.0,3861.0,26.0,5220.0,194525.0,166735.0,0.0,0.0
3,AK,2021-3,732673,62559.0,299.0,104247.0,1954197.0,601225.0,407233.0,248711.0,163100.0,0.0,4713.0,20.0,5328.0,212166.0,81976.0,163100.0,0.0
4,AK,2021-4,732673,67598.0,330.0,109932.0,2152524.0,769205.0,552792.0,305395.0,256564.0,0.0,5039.0,31.0,5685.0,198327.0,56684.0,93464.0,0.0
5,AK,2021-5,732673,69627.0,352.0,112288.0,2299052.0,806335.0,612275.0,337795.0,287850.0,0.0,2029.0,22.0,2356.0,146528.0,32400.0,31286.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,WY,2021-8,578803,75136.0,858.0,62119.0,865487.0,586155.0,478281.0,262132.0,223590.0,0.0,10009.0,82.0,6378.0,59414.0,20522.0,11360.0,0.0
685,WY,2021-9,578803,90602.0,996.0,72700.0,949582.0,658215.0,520777.0,280635.0,240814.0,0.0,15466.0,138.0,10581.0,84095.0,18503.0,17224.0,0.0
686,WY,2021-10,578803,102926.0,1174.0,80967.0,1018072.0,742965.0,569168.0,294284.0,254028.0,30404.0,12324.0,178.0,8267.0,68490.0,13649.0,13214.0,30404.0
687,WY,2021-11,578803,111089.0,1428.0,85935.0,1076921.0,813665.0,632809.0,310481.0,263763.0,69165.0,8163.0,254.0,4968.0,58849.0,16197.0,9735.0,38761.0


In [12]:
final_monthly_set1_df.to_csv("Updated_Final_COVID_Data.csv")